In [10]:
import wrangle as w 
import pandas as pd
import numpy as np
import scipy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import regex as re
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import export_text
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
df = w.wrangle_df()

clean.csv not detected.
Acquiring and Preparing Data
Using cached CSV
clean.csv not detected.
processing capstone.csv
clean.csv ready for future use


In [3]:
df.columns

Index(['title', 'success', 'success_rating', 'genres', 'cast_actor_1',
       'cast_actor_2', 'cast_actor_3', 'total_n_cast', 'budget', 'revenue',
       'profit_amount', 'vote_average', 'vote_count', 'production_companies',
       'production_countries', 'overview', 'popularity', 'runtime',
       'profitable', 'release_date', 'release_year', 'imdb_id',
       'is_genre_adventure', 'is_genre_horror', 'is_genre_drama',
       'is_genre_scifi', 'is_genre_romance', 'is_genre_thriller',
       'is_genre_crime', 'is_genre_comedy', 'is_genre_animation',
       'is_genre_action', 'is_genre_mystery', 'is_genre_fantasy',
       'is_genre_documentary'],
      dtype='object')

In [19]:
df.production_companies

id
5                                   MiramaxFilms ABandApart
6         UniversalPictures LargoEntertainment JVCEntert...
11             Lucasfilm TwentiethCenturyFoxFilmCorporation
12                                    PixarAnimationStudios
13                                        ParamountPictures
                                ...                        
433356                                  EasternSunriseFilms
434119                                      CJEntertainment
437031    TornasolFilms CastafioreFilms RadioTelevisiónE...
440642                                   BazelevsProduction
443319                    ScottFreeProductions CinelouFilms
Name: production_companies, Length: 6893, dtype: object

In [20]:
re.search(r'[A-Z][a-z]+', str(df.production_companies))

<regex.Match object; span=(39, 46), match='Miramax'>

In [24]:
df['production_companies'].split(r'[A-Z][a-z]+')

AttributeError: 'Series' object has no attribute 're'

In [29]:
df['production_company'] = df['production_companies'].str.split().str.get(0)

In [31]:
df

,title,success,success_rating,genres,cast_actor_1,cast_actor_2,cast_actor_3,total_n_cast,budget,revenue,...,is_genre_thriller,is_genre_crime,is_genre_comedy,is_genre_animation,is_genre_action,is_genre_mystery,is_genre_fantasy,is_genre_documentary,company,production_company
id,,,,,,,,,,,,,,,,,,,,,
5,Four Rooms,0,3.493750,Crime Comedy,Tim Roth,Antonio Banderas,Jennifer Beals,24.0,4000000.0,4300000.0,...,0,1,1,0,0,0,0,0,NaN,MiramaxFilms
6,Judgment Night,0,3.883820,Action Thriller Crime,Emilio Estevez,Cuba Gooding Jr.,Denis Leary,15.0,10000000.0,12136938.0,...,1,1,0,0,1,0,0,0,NaN,UniversalPictures
11,Star Wars,1,285.487448,Adventure Action Science Fiction,Mark Hamill,Harrison Ford,Carrie Fisher,106.0,11000000.0,775398007.0,...,0,0,0,0,1,0,0,0,NaN,Lucasfilm
12,Finding Nemo,1,38.013564,Animation Family,Albert Brooks,Ellen DeGeneres,Alexander Gould,24.0,94000000.0,940335536.0,...,0,0,0,1,0,0,0,0,NaN,PixarAnimationStudios
13,Forrest Gump,1,50.537748,Comedy Drama Romance,Tom Hanks,Robin Wright,Gary Sinise,66.0,55000000.0,677945399.0,...,0,0,1,0,0,0,0,0,NaN,ParamountPictures
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433356,The Ottoman Lieutenant,0,0.019579,Romance Drama War,Michiel Huisman,Hera Hilmar,Josh Hartnett,24.0,40000000.0,240978.0,...,0,0,0,0,0,0,0,0,NaN,EasternSunriseFilms
434119,Confidential Assignment,1,20.411972,Action Drama Comedy,Hyun Bin,Yoo Hae-jin,Kim Joo-hyuk,7.0,8520000.0,56100000.0,...,0,0,1,0,1,0,0,0,NaN,CJEntertainment
437031,Rescue Under Fire,0,1.525000,War,Ariadna Gil,Roberto Álamo,Antonio Garrido,13.0,10000000.0,5000000.0,...,0,0,0,0,0,0,0,0,NaN,TornasolFilms


In [27]:
df

,title,success,success_rating,genres,cast_actor_1,cast_actor_2,cast_actor_3,total_n_cast,budget,revenue,...,is_genre_romance,is_genre_thriller,is_genre_crime,is_genre_comedy,is_genre_animation,is_genre_action,is_genre_mystery,is_genre_fantasy,is_genre_documentary,company
id,,,,,,,,,,,,,,,,,,,,,
5,Four Rooms,0,3.493750,Crime Comedy,Tim Roth,Antonio Banderas,Jennifer Beals,24.0,4000000.0,4300000.0,...,0,0,1,1,0,0,0,0,0,NaN
6,Judgment Night,0,3.883820,Action Thriller Crime,Emilio Estevez,Cuba Gooding Jr.,Denis Leary,15.0,10000000.0,12136938.0,...,0,1,1,0,0,1,0,0,0,NaN
11,Star Wars,1,285.487448,Adventure Action Science Fiction,Mark Hamill,Harrison Ford,Carrie Fisher,106.0,11000000.0,775398007.0,...,0,0,0,0,0,1,0,0,0,NaN
12,Finding Nemo,1,38.013564,Animation Family,Albert Brooks,Ellen DeGeneres,Alexander Gould,24.0,94000000.0,940335536.0,...,0,0,0,0,1,0,0,0,0,NaN
13,Forrest Gump,1,50.537748,Comedy Drama Romance,Tom Hanks,Robin Wright,Gary Sinise,66.0,55000000.0,677945399.0,...,1,0,0,1,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433356,The Ottoman Lieutenant,0,0.019579,Romance Drama War,Michiel Huisman,Hera Hilmar,Josh Hartnett,24.0,40000000.0,240978.0,...,1,0,0,0,0,0,0,0,0,NaN
434119,Confidential Assignment,1,20.411972,Action Drama Comedy,Hyun Bin,Yoo Hae-jin,Kim Joo-hyuk,7.0,8520000.0,56100000.0,...,0,0,0,1,0,1,0,0,0,NaN
437031,Rescue Under Fire,0,1.525000,War,Ariadna Gil,Roberto Álamo,Antonio Garrido,13.0,10000000.0,5000000.0,...,0,0,0,0,0,0,0,0,0,NaN


In [15]:
## Function to clean the names
#def clean_companies(column):
#    # Search for opening bracket in the name followed by
#    # any characters repeated any number of times
#    if re.search(r'[A-Z][a-z]+', column):
#  
#        # Extract the position of beginning of pattern
#        pos = re.search(r'[A-Z][a-z]+', column)
#  
#        # return the cleaned name
#        return column[:pos]
#  
#    else:
#        # if clean up needed return the same name
#        return column
#          
## Updated the city columns
##df['production_companies'] = df['production_companies'].apply(Clean_names)

In [4]:
df.dtypes

title                           object
success                          int64
success_rating                 float64
genres                          object
cast_actor_1                    object
cast_actor_2                    object
cast_actor_3                    object
total_n_cast                   float64
budget                         float64
revenue                        float64
profit_amount                  float64
vote_average                   float64
vote_count                     float64
production_companies            object
production_countries            object
overview                        object
popularity                     float64
runtime                        float64
profitable                       int64
release_date            datetime64[ns]
release_year                     int64
imdb_id                         object
is_genre_adventure               int64
is_genre_horror                  int64
is_genre_drama                   int64
is_genre_scifi           

The data types are as they should be, as far as I know. 

In [5]:
train, validate, test = w.train_validate_test_split(df)

In [7]:
print('train:', train.shape)
print('validate:', validate.shape)
print('test:', test.shape)

train: (3859, 35)
validate: (1655, 35)
test: (1379, 35)
